# go-database/sql接口

Go没有内置的驱动支持任何的数据库，但是Go定义了database/sql接口，用户可以基于驱动接口开发相应数据库的驱动.这样做有一个好处，只要是按照标准接口开发的代码， 以后需要迁移数据库时，不需要任何修改。

### sql.Register

这个是database/sql中的一个函数, 用来注册数据库驱动，当第三方开发者开发数据库驱动时，都会实现init函数，在init里面会调用这个Register(name string, driver driver.Driver)完成本驱动的注册。

database/sql的注册函数可以同时注册多个数据库驱动，只要不重复。

### driver.Driver

Driver是一个数据库驱动的接口，他定义了一个method： Open(name string)，这个方法返回一个数据库的Conn接口。

返回的Conn只能用来进行一次goroutine的操作，也就是说不能把这个Conn应用于Go的多个goroutine里面, 会出现错误.

### driver.Conn

Conn是一个数据库连接的接口定义，他定义了一系列方法，这个Conn只能应用在一个goroutine里面，不能使用在多个goroutine里面

### driver.Stmt

Stmt是一种准备好的状态，和Conn相关联，而且只能应用于一个goroutine中，不能应用于多个goroutine。

### driver.Tx

事务处理一般就两个过程，递交或者回滚。数据库驱动里面也只需要实现这两个函数就可以
```
type Tx interface {
	Commit() error
	Rollback() error
}
```
这两个函数一个用来递交一个事务，一个用来回滚事务。

### driver.Execer

这是一个Conn可选择实现的接口, 如果这个接口没有定义，那么在调用DB.Exec,就会首先调用Prepare返回Stmt，然后执行Stmt的Exec，然后关闭Stmt。

### driver.Result

这个是执行Update/Insert等操作返回的结果接口定义,
```
type Result interface {
	LastInsertId() (int64, error)
	RowsAffected() (int64, error)
}
```
LastInsertId函数返回由数据库执行插入操作得到的自增ID号。

RowsAffected函数返回query操作影响的数据条目数。

### driver.Rows

Rows是执行查询返回的结果集接口定义
```
type Rows interface {
	Columns() []string
	Close() error
	Next(dest []Value) error
}
```
Columns函数返回查询数据库表的字段信息，这个返回的slice和sql查询的字段一一对应，而不是返回整个表的所有字段。

Close函数用来关闭Rows迭代器。

Next函数用来返回下一条数据，把数据赋值给dest。dest里面的元素必须是driver.Value的值除了string，返回的数据里面所有的string都必须要转换成[]byte。如果最后没数据了，Next函数最后返回io.EOF。

### driver.RowsAffected

RowsAffected其实就是一个int64的别名，但是他实现了Result接口，用来底层实现Result的表示方式
```
type RowsAffected int64

func (RowsAffected) LastInsertId() (int64, error)

func (v RowsAffected) RowsAffected() (int64, error)
```

### driver.Value

Value其实就是一个空接口，他可以容纳任何的数据

`type Value interface{}`

drive的Value是驱动必须能够操作的Value，Value要么是nil，要么是下面的任意一种
```
int64
float64
bool
[]byte
string   [*]除了Rows.Next返回的不能是string.
time.Time
```

### driver.ValueConverter

ValueConverter接口定义了如何把一个普通的值转化成driver.Value的接口
```
type ValueConverter interface {
	ConvertValue(v interface{}) (Value, error)
}
```
在开发的数据库驱动包里面实现这个接口的函数在很多地方会使用到，这个ValueConverter有很多好处：

* 转化driver.value到数据库表相应的字段，例如int64的数据如何转化成数据库表uint16字段
* 把数据库查询结果转化成driver.Value值
* 在scan函数里面如何把driver.Value值转化成用户定义的值

### driver.Valuer

Valuer接口定义了返回一个driver.Value的方式
```
type Valuer interface {
	Value() (Value, error)
}
```
很多类型都实现了这个Value方法，用来自身与driver.Value的转化。

通过上面的讲解，你应该对于驱动的开发有了一个基本的了解，一个驱动只要实现了这些接口就能完成增删查改等基本操作了，剩下的就是与相应的数据库进行数据交互等细节问题了。

### database/sql

database/sql在database/sql/driver提供的接口基础上定义了一些更高阶的方法，用以简化数据库操作,同时内部还建议性地实现一个conn pool。
```
type DB struct {
	driver 	 driver.Driver
	dsn    	 string
	mu       sync.Mutex // protects freeConn and closed
	freeConn []driver.Conn
	closed   bool
}
```
我们可以看到Open函数返回的是DB对象，里面有一个freeConn，它就是那个简易的连接池。它的实现相当简单或者说简陋，就是当执行db.prepare -> db.prepareDC的时候会defer dc.releaseConn，然后调用db.putConn，也就是把这个连接放入连接池，每次调用db.conn的时候会先判断freeConn的长度是否大于0，大于0说明有可以复用的conn，直接拿出来用就是了，如果不大于0，则创建一个conn，然后再返回之。